## The game:
I want to combine a shooter style game with hangman where the player must shoot the letter which they want to guess. This adds a layer of complexity and fun to the normal hangman.

For this project, I will be using ```pygame``` to show images on a screen.

## Structure

I will be using classes to abstract various parts of the game.

- ```Hangman``` Class: Draws the gallows and body parts based on flags.
    - ```__init__```: Initializes the Hangman object, takes parameters ```x, y``` for the position of the gallows on the screen.
    - ```draw```: Draws the gallows and parts of the Hangman (basic parts such as head, body, arms, and legs, as well as additional parts like hands and feet) if their respective flags are set to True.

- ```MysteryWord``` Class: Manages the secret word and drawing of guessed letters.
    - ```__init__```: Initializes the graphics for the secret word. Takes parameters ```word, pos, font_size, color```, with ```font_size``` and ```color``` defaulted to 40 and white respectively.
    - ```guess```: Adds a letter to the set of guessed letters, which results in that letter being drawn on the screen.
    - ```draw```: Renders the mystery word with correctly guessed letters shown and unguessed letters as underscores, centered at the specified position.

- ```Shooter``` Class: Represents the player-controlled sprite that fires bullets.
    - ```__init__```: Initializes the shooter, creates a polygonal image, and positions it (centered horizontally on the screen).
    - ```move```: Moves the shooter horizontally by a specified amount.
    - ```getPos```: Returns the current center position of the shooter, used for spawning bullets.

- ```Bullet``` Class: Represents a bullet fired by the shooter.
    - ```__init__```: Initializes the bullet with a specified position, setting its radius, color, and upward velocity.
    - ```update```: Moves the bullet upward each frame; if the bullet leaves the main game area (above 600 pixels), it is removed from the game.

- ```LetterSprite``` Class: Represents a letter bouncing around in the main game area.
    - ```__init__```: Initializes the letter sprite by setting the displayed letter, its starting position, and a random non-zero velocity.
    - ```update```: Moves the letter sprite based on its velocity and makes it bounce off the screen boundaries (horizontal boundaries of 0 and the screen width, vertical boundaries limited to the main game area).

- ```Game``` Class: Integrates all components and handles overall game logic, input, and display.
    - ```__init__```: 
        - Sets up the display (1200x800 pixels) and divides it into three areas:
            - Top 550 pixels: Main game area (shooter, bullets, and bouncing letters).
            - 550–700 pixels: Hangman area (gallows and hangman parts).
            - 700–800 pixels: Mystery word area (display of the secret word).
        - Creates and initializes sprite groups for the shooter, bullets, and letters.
        - Instantiates the ```Hangman``` and ```MysteryWord``` objects.
        - Sets initial game state variables such as guessed letters, incorrect guess count, and the order in which hangman parts are revealed.
    - ```_handle_input```: Processes user input, including:
        - Firing bullets with the space bar.
        - Moving the shooter with arrow keys.
        - Making manual letter guesses via keyboard.
    - ```_process_game_logic```: 
        - Updates sprite positions.
        - Checks for collisions between bullets and letter sprites; on collision, marks the corresponding letter as chosen.
        - Updates the hangman based on incorrect guesses.
        - Checks win conditions (all letters in the mystery word have been guessed) and loss conditions (all hangman parts are revealed).
    - ```_draw```: Renders the current game state by:
        - Drawing the main game sprites in the top 600 pixels.
        - Drawing separators between the game areas.
        - Drawing the hangman in the middle area.
        - Drawing the mystery word in the bottom area.
    - ```_show_end_screen```: Displays a win or loss screen when the game is over and waits for a key press before quitting.
    - ```run```: Contains the main game loop that continuously:
        - Handles input.
        - Processes game logic.
        - Draws the current game state.
        - Checks for win/loss conditions to potentially exit the loop and display the end screen.


Note: code is designed to be run in a folder with the file ```words.txt``` in the same folder.
Note: I had some help from ChatGPT when writing this code (as it knows pygame much better than me).

In [ ]:
import pygame
import random

# Define some colors.
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)


# -------------------------------------------------------------------------
# Hangman Class: Draws the gallows and body parts based on flags.
# -------------------------------------------------------------------------
class Hangman:
    def __init__(self, x, y):
        """
        :param x: The x-coordinate for drawing the gallows.
        :param y: The y-coordinate (base) for the gallows.
        """
        self.x = x
        self.y = y
        # Basic body parts.
        self.head = False
        self.body = False
        self.left_arm = False
        self.right_arm = False
        self.left_leg = False
        self.right_leg = False
        # Additional parts.
        self.left_hand = False
        self.right_hand = False
        self.left_foot = False
        self.right_foot = False

    def draw(self, surface):
        # Draw the gallows.
        pygame.draw.line(surface, WHITE, (self.x, self.y), (self.x, self.y - 100), 5)
        pygame.draw.line(surface, WHITE, (self.x, self.y - 100), (self.x + 50, self.y - 100), 5)
        pygame.draw.line(surface, WHITE, (self.x + 50, self.y - 100), (self.x + 50, self.y - 90), 5)

        # Draw basic body parts if enabled.
        if self.head:
            pygame.draw.circle(surface, WHITE, (self.x + 50, self.y - 80), 10, 3)
        if self.body:
            pygame.draw.line(surface, WHITE, (self.x + 50, self.y - 70), (self.x + 50, self.y - 40), 3)
        if self.left_arm:
            pygame.draw.line(surface, WHITE, (self.x + 50, self.y - 60), (self.x + 40, self.y - 55), 3)
        if self.right_arm:
            pygame.draw.line(surface, WHITE, (self.x + 50, self.y - 60), (self.x + 60, self.y - 55), 3)
        if self.left_leg:
            pygame.draw.line(surface, WHITE, (self.x + 50, self.y - 40), (self.x + 40, self.y - 30), 3)
        if self.right_leg:
            pygame.draw.line(surface, WHITE, (self.x + 50, self.y - 40), (self.x + 60, self.y - 30), 3)

        # Draw additional parts if enabled.
        if self.left_hand:
            pygame.draw.circle(surface, WHITE, (self.x + 40, self.y - 55), 3)
        if self.right_hand:
            pygame.draw.circle(surface, WHITE, (self.x + 60, self.y - 55), 3)
        if self.left_foot:
            pygame.draw.circle(surface, WHITE, (self.x + 40, self.y - 30), 3)
        if self.right_foot:
            pygame.draw.circle(surface, WHITE, (self.x + 60, self.y - 30), 3)


# -------------------------------------------------------------------------
# MysteryWord Class: Manages the secret word and drawing of guessed letters.
# -------------------------------------------------------------------------
class MysteryWord:
    def __init__(self, word, pos, font_size=40, color=WHITE):
        """
        :param word: The secret word (a string).
        :param pos: The (x, y) position (centered) where the mystery word is drawn.
        :param font_size: Font size for drawing the word.
        :param color: Color of the text.
        """
        self.word = word.upper()
        self.guessed = set()  # Letters that have been correctly guessed.
        self.pos = pos
        self.font = pygame.font.SysFont('Arial', font_size)
        self.color = color

    def guess(self, letter):
        """Add a letter to the set of correctly guessed letters."""
        self.guessed.add(letter.upper())

    def draw(self, surface):
        """Draw the mystery word with revealed letters and underscores for unguessed letters."""
        display_text = ""
        for letter in self.word:
            if letter in self.guessed:
                display_text += letter + " "
            else:
                display_text += "_ "
        text_surface = self.font.render(display_text.strip(), True, self.color)
        text_rect = text_surface.get_rect(center=self.pos)
        surface.blit(text_surface, text_rect)


# -------------------------------------------------------------------------
# Shooter Class: The player-controlled sprite that fires bullets.
# -------------------------------------------------------------------------
class Shooter(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.size = 20
        self.image = pygame.Surface((self.size, self.size), pygame.SRCALPHA)
        pygame.draw.polygon(self.image, GREEN, [(0, 0), (self.size / 2, self.size), (self.size, 0)])
        self.rect = self.image.get_rect()
        # Center shooter in a 1200-wide screen (x = 600).
        self.rect.x, self.rect.y = 600, 10

    def move(self, amnt):
        self.rect.x += amnt
        self.rect.x = max(0, min(self.rect.x, 1170))

    def getPos(self):
        return self.rect.center


# -------------------------------------------------------------------------
# Bullet Class: Moves upward and is removed if it leaves the main game area.
# -------------------------------------------------------------------------
class Bullet(pygame.sprite.Sprite):
    def __init__(self, pos):
        super().__init__()
        self.radius = 5
        self.image = pygame.Surface((self.radius * 2, self.radius * 2), pygame.SRCALPHA)
        pygame.draw.circle(self.image, BLUE, (self.radius, self.radius), self.radius)
        self.rect = self.image.get_rect()
        self.rect.x, self.rect.y = pos
        self.velocity = 10

    def update(self):
        self.rect.y += self.velocity
        # Remove the bullet if it goes beyond the main game area (top 600 pixels).
        if self.rect.y > 550:
            self.kill()


# -------------------------------------------------------------------------
# LetterSprite Class: Represents a letter bouncing in the main game area.
# -------------------------------------------------------------------------
class LetterSprite(pygame.sprite.Sprite):
    def __init__(self, letter, pos, color=WHITE, font_size=30, velocity=None):
        super().__init__()
        self.letter = letter
        self.font = pygame.font.SysFont('Arial', font_size)
        self.image = self.font.render(letter, True, color)
        self.rect = self.image.get_rect(center=pos)
        # Choose a random non-zero velocity if none is provided.
        if velocity is None:
            vx = random.choice([-3, -2, -1, 1, 2, 3])
            vy = random.choice([-3, -2, -1, 1, 2, 3])
            self.velocity = [vx, vy]
        else:
            self.velocity = list(velocity)

    def update(self):
        self.rect.x += self.velocity[0]
        self.rect.y += self.velocity[1]
        # Bounce off the left/right boundaries (0 and 1200).
        if self.rect.left < 0 or self.rect.right > 1200:
            self.velocity[0] = -self.velocity[0]
        # Bounce off the top/bottom boundaries of the main game area (0-550 pixels).
        if self.rect.top < 0 or self.rect.bottom > 550:
            self.velocity[1] = -self.velocity[1]


# -------------------------------------------------------------------------
# Game Class: Integrates the shooter, bullet/letter sprites, hangman, mystery word,
# and displays win/loss screens.
# -------------------------------------------------------------------------
class Game:
    def __init__(self):
        pygame.init()
        # Expanded display: 1200x800 pixels.
        # - Top 550 pixels: Main game area.
        # - 5500-700 pixels: Hangman area.
        # - 700-800 pixels: Mystery word area.
        self.size = (1200, 800)
        self.screen = pygame.display.set_mode(self.size)
        pygame.display.set_caption("ShootOrHang")
        self.clock = pygame.time.Clock()

        # Create the shooter.
        self.shooter = Shooter()
        self.all_sprites = pygame.sprite.Group()
        self.all_sprites.add(self.shooter)

        # Create groups for bullets and letter sprites.
        self.bullets = pygame.sprite.Group()
        self.letters = pygame.sprite.Group()

        # Create letter sprites (example for the word "HELLO").
        letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        for i, letter in enumerate(letters):
            pos = (random.randrange(20, 1180), random.randrange(20, 500))
            letter_sprite = LetterSprite(letter, pos)
            self.all_sprites.add(letter_sprite)
            self.letters.add(letter_sprite)

        # Create the Hangman structure (displayed in the 550-700 pixel area).
        self.hangman = Hangman(50, 680)

        # Create the Mystery Word (displayed in the 700-800 pixel area).
        with open("words.txt") as words:
            lines = words.readlines()
            word = ""
            while word == "":
                idx = random.randrange(len(lines))
                if len(lines[idx].upper()) < 13:
                    word = lines[idx].upper()
            self.mystery_word = MysteryWord(word.strip(), pos=(600, 750))

        # Track chosen letters (via keyboard or bullet collisions).
        self.guessed_letters = set()
        self.incorrect_guess_count = 0
        # Parts order (10 parts now).
        self.parts_order = [
            "head", "body", "left_arm", "right_arm",
            "left_leg", "right_leg", "left_hand", "right_hand",
            "left_foot", "right_foot"
        ]

        # Game state flags.
        self.game_over = False
        self.win = False

    def _handle_input(self):
        # If the game is over, we don't process normal input.
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            elif not self.game_over and event.type == pygame.KEYDOWN:
                # Space bar: Fire a bullet.
                if event.key == pygame.K_SPACE:
                    spos = self.shooter.getPos()
                    bullet = Bullet((spos[0] - 4, spos[1] + 20))
                    self.all_sprites.add(bullet)
                    self.bullets.add(bullet)

        # Allow continuous movement with key holds.
        keys = pygame.key.get_pressed()
        if not self.game_over:
            if keys[pygame.K_RIGHT]:
                self.shooter.move(10)
            if keys[pygame.K_LEFT]:
                self.shooter.move(-10)

    def _process_game_logic(self):
        # Only update game logic if the game is not over.
        if not self.game_over:
            self.all_sprites.update()

            # Check for collisions between bullets and letter sprites.
            collisions = pygame.sprite.groupcollide(self.bullets, self.letters, True, True)
            if collisions:
                for bullet, letters_hit in collisions.items():
                    for letter_sprite in letters_hit:
                        chosen_letter = letter_sprite.letter.upper()
                        if chosen_letter not in self.guessed_letters:
                            self.guessed_letters.add(chosen_letter)
                            if chosen_letter in self.mystery_word.word:
                                self.mystery_word.guess(chosen_letter)
                                print(f"Correct guess by bullet collision: {chosen_letter}")
                            else:
                                print(f"Incorrect guess by bullet collision: {chosen_letter}")
                                self.incorrect_guess_count += 1
                                if self.incorrect_guess_count <= len(self.parts_order):
                                    part_name = self.parts_order[self.incorrect_guess_count - 1]
                                    setattr(self.hangman, part_name, True)

            # Bounce letter sprites off one another.
            letter_list = list(self.letters)
            for i in range(len(letter_list)):
                for j in range(i + 1, len(letter_list)):
                    l1 = letter_list[i]
                    l2 = letter_list[j]
                    if l1.rect.colliderect(l2.rect):
                        l1.velocity, l2.velocity = l2.velocity, l1.velocity

            # Check win condition: all letters in the mystery word are guessed.
            if set(self.mystery_word.word).issubset(self.mystery_word.guessed):
                self.game_over = True
                self.win = True
            # Check loss condition: maximum incorrect guesses reached.
            elif self.incorrect_guess_count >= len(self.parts_order):
                self.game_over = True
                self.win = False

    def _draw(self):
        self.screen.fill(BLACK)
        # Draw main game sprites (top 600 pixels).
        self.all_sprites.draw(self.screen)
        # Separator between game area and hangman area.
        pygame.draw.line(self.screen, WHITE, (0, 550), (1200, 550), 2)
        # Draw the hangman (600-700 pixels).
        self.hangman.draw(self.screen)
        # Separator between hangman and mystery word areas.
        pygame.draw.line(self.screen, WHITE, (0, 700), (1200, 700), 2)
        # Draw the mystery word (700-800 pixels).
        self.mystery_word.draw(self.screen)

        pygame.display.flip()
        self.clock.tick(60)

    def _show_end_screen(self):
        # Display win or loss message.
        self.screen.fill(BLACK)
        font = pygame.font.SysFont('Arial', 60)
        if self.win:
            message = "You Win!"
        else:
            message = "Game Over! You Lose!"
        text_surface = font.render(message, True, WHITE)
        text_rect = text_surface.get_rect(center=(600, 400))
        self.screen.blit(text_surface, text_rect)
        pygame.display.flip()

        # Wait for any key press before quitting.
        waiting = True
        while waiting:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    waiting = False
                    pygame.quit()
                    quit()
                elif event.type == pygame.KEYDOWN:
                    waiting = False

    def run(self):
        while True:
            self._handle_input()
            self._process_game_logic()
            self._draw()
            if self.game_over:
                self._show_end_screen()
                break


if __name__ == "__main__":
    Game().run()
